# Projeto de Turismo com LangChain - Passo 1: Ingestão de Dados

Este notebook é responsável por preparar e enviar nossa base de conhecimento para o Pinecone. O processo consiste em:
1. Carregar as chaves de API.
2. Ler os arquivos de texto da pasta `/data`.
3. Dividir os textos em "chunks" (pedaços).
4. Gerar "embeddings" (vetores numéricos) para cada chunk.
5. Criar um índice no Pinecone (se não existir) e enviar os dados.

## 0. Pré-requisitos

Certifique-se de que as bibliotecas necessárias estão instaladas. Se não estiverem, execute a célula abaixo (removendo o `#` inicial).

# !pip install langchain langchain-community langchain-groq langchain-pinecone pinecone-client python-dotenv sentence-transformers torch

## 1. Carregar Bibliotecas e Variáveis de Ambiente

Primeiro, importamos todas as ferramentas que vamos usar e carregamos nossas chaves secretas do arquivo `.env`. É fundamental que o arquivo `.env` esteja na mesma pasta que este notebook.

In [1]:
import os
import time
from dotenv import load_dotenv

from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore

from pinecone import Pinecone, ServerlessSpec

# Carrega as variáveis do arquivo .env
load_dotenv()

# Pega as variáveis do ambiente
pinecone_api_key = os.getenv("PINECONE_API_KEY")
index_name = os.getenv("PINECONE_INDEX_NAME")

# Verificação de segurança
if not pinecone_api_key or not index_name:
    print("Erro: Verifique se as variáveis PINECONE_API_KEY e PINECONE_INDEX_NAME estão no arquivo .env")
else:
    print("Variáveis de ambiente carregadas com sucesso!")

/home/weriks/anaconda3/envs/sistema_turismo_langchain/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Variáveis de ambiente carregadas com sucesso!


/home/weriks/anaconda3/envs/sistema_turismo_langchain/lib/python3.13/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


## 2. Carregar os Documentos da Pasta `/data`

Agora, vamos usar o `DirectoryLoader` do LangChain para encontrar e carregar todos os arquivos `.txt` que estão na nossa pasta de dados.

In [2]:
print("--- Carregando documentos da pasta /data ---")

loader = DirectoryLoader(
    "data/",
    glob="*.txt",  # Padrão para buscar apenas arquivos .txt
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}
)

documents = loader.load()

print(f"Carregados {len(documents)} documentos.")
# Para inspecionar um documento, você pode descomentar a linha abaixo:
print(documents[0].page_content[:500]) # Mostra os primeiros 500 caracteres do primeiro documento

--- Carregando documentos da pasta /data ---
Carregados 2 documentos.
 
# Informações Gerais sobre Paris, França

Paris, a capital da França, é um centro global de arte, moda, gastronomia e cultura. Conhecida como a "Cidade Luz", é famosa por seus monumentos icônicos, museus de classe mundial, charmosos cafés e alta costura. O rio Sena corta a cidade, e suas margens são repletas de pontos de interesse.

## Principais Pontos Turísticos

### Torre Eiffel
- **Descrição:** O monumento mais emblemático de Paris, construído por Gustave Eiffel para a Exposição Universal 


## 3. Dividir os Documentos em Chunks

Documentos longos são ineficientes para buscas de similaridade. Por isso, nós os quebramos em pedaços menores (chunks). O `RecursiveCharacterTextSplitter` é inteligente e tenta manter parágrafos e sentenças coesos.

In [3]:
print("--- Dividindo os documentos em chunks ---")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Tamanho máximo de cada chunk em caracteres
    chunk_overlap=200,  # Sobreposição entre chunks para não perder o contexto
    length_function=len
)

docs_split = text_splitter.split_documents(documents)

print(f"Documentos divididos em {len(docs_split)} chunks.")
# Para inspecionar um chunk, você pode descomentar a linha abaixo:
print("\n--- Exemplo de Chunk ---")
print(docs_split[5].page_content)

--- Dividindo os documentos em chunks ---
Documentos divididos em 11 chunks.

--- Exemplo de Chunk ---
# Informações Gerais sobre o Rio de Janeiro, Brasil

O Rio de Janeiro, conhecido como a "Cidade Maravilhosa", é um dos principais destinos turísticos do Brasil. É famoso mundialmente por suas praias deslumbrantes como Copacabana e Ipanema, a estátua do Cristo Redentor no topo do Corcovado, o Pão de Açúcar e o animado Carnaval. A cidade oferece uma mistura única de metrópole urbana e natureza exuberante, com montanhas e florestas tropicais em meio à paisagem urbana.

## Principais Pontos Turísticos


## 4. Inicializar o Modelo de Embeddings

A "mágica" do RAG acontece aqui. Um modelo de embedding transforma o texto de cada chunk em um vetor (uma lista de números). Textos com significados parecidos terão vetores próximos no espaço vetorial. Usaremos um modelo gratuito e de alta performance da Hugging Face.

In [4]:
print("--- Inicializando o modelo de embeddings ---")
print("Aguarde, isso pode levar um tempo para baixar o modelo na primeira execução...")

# Modelo popular e com bom desempenho
model_name = "sentence-transformers/all-MiniLM-L6-v2" 
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# O modelo 'all-MiniLM-L6-v2' gera vetores de dimensão 384.
# Guardamos este valor, pois é essencial para criar o índice no Pinecone.
embedding_dimension = 384

print("\nModelo de embeddings inicializado.")

/tmp/ipykernel_376653/3296105770.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)


--- Inicializando o modelo de embeddings ---
Aguarde, isso pode levar um tempo para baixar o modelo na primeira execução...

Modelo de embeddings inicializado.


## 5. Conectar e Configurar o Pinecone

Com os dados prontos e o modelo de embedding carregado, vamos preparar nossa base de dados vetorial. O código abaixo se conecta ao Pinecone e verifica se o nosso índice já existe. Se não existir, ele o cria.

In [5]:
print(f"--- Conectando ao Pinecone e preparando o índice '{index_name}' ---")

pc = Pinecone(api_key=pinecone_api_key)

# Verifica se o índice já existe
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    print(f"Índice '{index_name}' não encontrado. Criando um novo...")
    pc.create_index(
        name=index_name,
        dimension=embedding_dimension, # A dimensão DEVE ser a mesma do modelo de embedding
        metric="cosine", # Métrica de similaridade ideal para texto
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1' # Região padrão e gratuita
        ) 
    )
    # Aguarda um momento para o índice ficar pronto
    while not pc.describe_index(index_name).status['ready']:
        print("Aguardando o índice ficar pronto...")
        time.sleep(1)
    print("Índice criado com sucesso!")
else:
    print(f"Índice '{index_name}' já existe. Conectando...")

--- Conectando ao Pinecone e preparando o índice 'sistema-turismo' ---
Índice 'sistema-turismo' já existe. Conectando...


## 6. Ingerir os Dados no Pinecone

Este é o passo final. A função `from_documents` do LangChain irá, para cada chunk:
1. Gerar o embedding usando o modelo que carregamos.
2. Enviar o chunk e seu embedding correspondente para o índice do Pinecone.

Este processo pode demorar alguns minutos, dependendo da quantidade de dados.

In [6]:
print("--- Iniciando a ingestão dos documentos no Pinecone ---")

PineconeVectorStore.from_documents(
    documents=docs_split,
    embedding=embeddings,
    index_name=index_name
)

print("\n--- Ingestão de dados concluída com sucesso! ---")

--- Iniciando a ingestão dos documentos no Pinecone ---

--- Ingestão de dados concluída com sucesso! ---


## Conclusão

Pronto! Se todas as células foram executadas com sucesso, seu índice no Pinecone agora contém o conhecimento sobre o Rio de Janeiro e Paris. O próximo passo é construir as cadeias (Chains) que irão consultar esses dados para responder às perguntas dos usuários.